In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import pickle
import re 

#read in data 
petdat = pd.read_csv('backup_petdata_Jan30.csv')

#print(petdat.head())
print(list(petdat.columns))

['Unnamed: 0', 'Unnamed: 0.1', 'id', 'organization_id', 'url', 'type', 'species', 'age', 'gender', 'size', 'coat', 'tags', 'name', 'description', 'photos', 'status', 'status_changed_at', 'published_at', 'distance', 'breeds.primary', 'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary', 'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered', 'attributes.house_trained', 'attributes.declawed', 'attributes.special_needs', 'attributes.shots_current', 'environment.children', 'environment.dogs', 'environment.cats', 'contact.email', 'contact.phone', 'contact.address.address1', 'contact.address.address2', 'contact.address.city', 'contact.address.state', 'contact.address.postcode', 'contact.address.country', '_links.self.href', '_links.type.href', '_links.organization.href', 'City', 'time_diff', 'target_hour', 'target_day', 'log_target_hour', 'top_ten', 'num_tags', 'num_photos', 'num_words_desc', 'bin_month_target', 'bin_week_target']


In [8]:
import sklearn
from sklearn.linear_model import LinearRegression

traindat = petdat[['age',
                   #'organization_id',
                   #'contact.address.state',
                   'size',
                   #'coat',
                   #'top_ten',
                   #'attributes.spayed_neutered',
                  # 'attributes.house_trained',
                   #'attributes.special_needs',
                  # 'attributes.shots_current',
                   #'environment.cats',
                   #'environment.children',
                   #'environment.dogs',
                   'log_target_hour']]

X = traindat.drop(columns = ['log_target_hour'])

#one hot coding for features 
cols_to_transform = ['age',
                     'organization_id',
                     'contact.address.state',
                     'size',
                     'coat',
                     'top_ten',
                     'attributes.spayed_neutered',
                     'attributes.house_trained',
                     'attributes.special_needs',
                     'attributes.shots_current',
                     'environment.cats',
                     'environment.children',
                     'environment.dogs']
X = pd.get_dummies(X, columns = cols_to_transform)

#fit lm
lm = LinearRegression()

lm.fit(X, traindat.log_target_hour)
print(lm.score(X, traindat.log_target_hour))


print('Coefficients: \n', lm.coef_)
pd.DataFrame({'features': X.columns, 'estimatedCoefficients': lm.coef_})[['features', 'estimatedCoefficients']]


0.28092996787890834
Coefficients: 
 [ 6.99789767e+10  6.99789767e+10  6.99789767e+10  6.99789767e+10
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11
 -1.17028470e+11 -1.17028470e+11 -1.17028470e+11 -1.17

,features,estimatedCoefficients
0,age_Adult,6.997898e+10
1,age_Baby,6.997898e+10
2,age_Senior,6.997898e+10
3,age_Young,6.997898e+10
4,organization_id_AR224,-1.170285e+11
...,...,...
798,environment.cats_True,-5.343437e-02
799,environment.children_False,2.962384e-01
800,environment.children_True,-6.557465e-02
801,environment.dogs_False,3.185558e-01
